In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload


import holodeck as holo
import holodeck.detstats as ds
from holodeck import plot
import h5py

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import os

 Calculate Library Detstats

In [ ]:
# hdf_name = '/Users/emigardiner/GWs/holodeck/output/2023-05-09-mbp-ss14_n40_r10_f20_d17.5_l5_p0/ss_lib.hdf5'
# detstats_path = '/Users/emigardiner/GWs/holodeck/output/2023-05-09-mbp-ss14_n40_r10_f20_d17.5_l5_p0/detstats'

# npsrs = 40
# sigma = 1e-7
# output_dir = detstats_path+'/ds2_psrs40_sigma1e-7'

In [ ]:
# vals = ds.detect_lib(hdf_name, output_dir, npsrs, sigma, 
#                      nskies=25, thresh=0.5, plot=True, debug=True)

# Read in Library 

In [ ]:
sspath = '/Users/emigardiner/GWs/holodeck/output/2023-05-09-mbp-ss15_n100_r30_f100_d15_l5_p0/'
hdfname = sspath+'ss_lib.hdf5'
ssfile = h5py.File(hdfname, 'r')
print(list(ssfile.keys()))
hc_ss = ssfile['hc_ss'][...]
hc_bg = ssfile['hc_bg'][...]
ssfile.close()

print('N,F,R,L:', hc_ss.shape) 
shape = hc_ss.shape
N, F, R, L = shape[0], shape[1], shape[2], shape[3]

# dsfile = np.load('/Users/emigardiner/GWs/holodeck/output/2023-05-09-mbp-ss15_n100_r30_f100_d15_l5_p0/detstats/dp4_psrs60_sigma1e-07/detstats.npz')
dsfile = np.load('/Users/emigardiner/GWs/holodeck/output/2023-05-12-mbp-ss16_n10_r10_f70_d12.5_l10_p0/detstats/ds2_psrs60_sigma5e-07/detstats.npz')
dp_ss = dsfile['dp_ss']
dp_bg = dsfile['dp_bg']
df_ss = dsfile['df_ss']
df_bg = dsfile['df_bg']
snr_ss = dsfile['snr_ss']
snr_bg = dsfile['snr_bg']
print('N,R,S:', dp_ss.shape)
S = dp_ss.shape[-1]

In [ ]:
print('snr_ss range:', np.min(snr_ss), np.max(snr_ss))

## SNR Histogram

In [ ]:
print(snr_ss.shape, snr_bg.shape)
print(snr_ss.min())

In [ ]:
flat_snr = snr_ss.flatten()
bins = np.geomspace(1e-7, flat_snr.max(), 100+1)
plt.hist(flat_snr, bins=bins)

plt.yscale('log')
plt.xscale('log')
plt.title('Histogram of SNRs for the 10th Loudest Sources in each Frequency Bin')
plt.ylabel('Number of Single Sources')
plt.xlabel('SNR_ss')

# Integration

Find Fe_bar

In [ ]:
Num = F*L
Fe_bar = ds._Fe_thresh(Num)
Fe_bar = np.float64(Fe_bar)

run_integration

In [ ]:
rho = np.geomspace(10**-3, 10**3, 10**3)
gamma = np.zeros_like(rho)

for rr in range(len(rho)):
    gamma[rr] = ds._gamma_of_rho(Fe_bar, rho[rr])

plot gamma(rho)

In [ ]:
def plot_gamma_vs_rho(rho, gamma, label=''):
    fig, ax = plot.figax(xlabel=r'$\rho$', ylabel=r'$\gamma(\rho)$')
    title = r'$\gamma(\rho) = \int_{\bar{\mathcal{F}}_e}^\infty \frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2} d\mathcal{F}_e$'
    ax.set_title(title)
    ax.scatter(rho, gamma, marker='x', s=2, label=label)
    ax.legend()
    return fig

fig = plot_gamma_vs_rho(rho, gamma, label='original')

Max gamma above peak rho

In [ ]:
arg_peak = np.nanargmax(gamma)
rho_peak = rho[arg_peak]
gamma_peak = np.max(gamma)
assert gamma_peak == gamma[arg_peak]

In [ ]:
def gamma_above_peak(gamma):
    arg_peak = np.nanargmax(gamma)
    gamma[arg_peak:] = gamma[arg_peak]
    return gamma

print('peak at index=%d, rho=%f, gamma=%f' % (arg_peak, rho_peak, gamma_peak))

gamma_maxed = np.copy(gamma)
gamma_maxed = gamma_above_peak(gamma_maxed)

fig = plot_gamma_vs_rho(rho, gamma_maxed, label='maxed above peak')

In [ ]:
def gamma_above_one(gamma):
    gamma[gamma>1.0] = 1.0
    return gamma

gamma_one = np.copy(gamma_maxed)
gamma_one = gamma_above_one(gamma_one)
fig = plot_gamma_vs_rho(rho, gamma_one, label='> 1.0 set to 1.0')

In [ ]:
plt.loglog(rho, gamma_maxed-gamma_one, 
           label='$\gamma_\mathrm{maxed} - \gamma_\mathrm{ones}$')
# plt.loglog(rho, gamma_maxed/gamma_one, 
#            label='$\gamma_\mathrm{maxed} / \gamma_\mathrm{ones}$')
plt.xlabel(r'$\rho$')
plt.ylabel('difference')
plt.legend()

## Save Grid

In [ ]:

# n(rho), n(Num) grid

outfile = '/Users/emigardiner/GWs/holodeck/output/rho_gamma_grids/rho_gamma_interp_grid_Num%d.npz' % (Num)
np.savez(outfile, rho_interp_grid=rho, gamma_interp_grid=gamma_one, Fe_bar=Fe_bar, Num=Num)

# Grid Interpolation

In [ ]:
from holodeck.constants import YR, GYR, MSOL
from holodeck import utils, sam, cosmo, sam_cython

In [ ]:
dur = 16.03*YR
cad = .2*YR
fobs_gw_cents = utils.nyquist_freqs(dur, cad)
fobs_gw_edges = utils.nyquist_freqs_edges(dur, cad)
fobs_orb_cents = fobs_gw_cents/2
fobs_orb_edges = fobs_gw_edges/2
sam = sam.Semi_Analytic_Model(mtot=(1.0e4*MSOL, 1.0e12*MSOL, 10), mrat=(1e-3, 1.0, 12), redz=(1e-3, 10.0, 14))
hard = holo.hardening.Fixed_Time_2PL_SAM(sam, 3*GYR)

In [ ]:
redz_final, diff_num = holo.sam_cython.dynamic_binary_number_at_fobs(
            fobs_orb_cents, sam, hard, cosmo
        )

## Check if interp grid exists

In [ ]:
Num = 500

gridname = '/Users/emigardiner/GWs/holodeck/output/rho_gamma_grids/rho_gamma_interp_grid_Num%d.npz' % (Num)
if os.path.exists(gridname):
    gridfile = np.load(gridname)
    rho_interp_grid = gridfile['rho_interp_grid']
    gamma_interp_grid = gridfile['gamma_interp_grid']
    Fe_bar = gridfile['Fe_bar']

### generate random rho's

In [ ]:
rho_exp = np.random.uniform(-6,6,10)
rho_rand = 10**rho_exp
print(rho_rand)

In [ ]:
rsort = np.argsort(rho_rand)
print(rsort)
print(rho_rand[rsort])

In [ ]:
print(np.min(rho_interp_grid))

In [ ]:
gamma_rand = sam_cython.gamma_of_rho_interp(rho_rand, rsort, rho_interp_grid, gamma_interp_grid)
print(gamma_rand)
print(gamma_rand[rsort])

In [ ]:
fig = plot_gamma_vs_rho(rho_interp_grid, gamma_interp_grid, label='interp_grid')
ax = fig.axes[0]
ax.scatter(rho_rand, gamma_rand, color='r', label='rand, interpolated in cython')
ax.legend()